# PROJ631 – Projet algorithmique
## Compte rendu Emma ORSET
## Compression de données par codage de Huffman

In [82]:
import os

## Etape 1 : Détermination de l’alphabet et des fréquences de caractères

Je récupère le fichier qui contient mon texte. Je l'ouvre puis je le lis. 
Cela créer une chaine de caractère.

In [83]:
# file_name = "extraitalice"
# file_name = "alice"
file_name = "textesimple"
with open(file_name +"/"+file_name + ".txt") as f:
    text = f.read()

print(text)

this is an example of a huffman tree


Je crée une liste qui contient chaque caractère de mon texte et je la trie selon l'ordre de la table ASCII.

In [84]:
list_sorted_label = []

for i in text:
    list_sorted_label.append(i)

list_sorted_label.sort()

print(list_sorted_label)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', 'a', 'a', 'a', 'a', 'e', 'e', 'e', 'e', 'f', 'f', 'f', 'h', 'h', 'i', 'i', 'l', 'm', 'm', 'n', 'n', 'o', 'p', 'r', 's', 's', 't', 't', 'u', 'x']


Je créer deux listes :
- list_label contient une ocurrence de chaque élement de la liste triée ci-dessus
- list_freq contient la fréquence de ces éléments

Cela crée mon alphabet

In [85]:
list_label = []
list_freq = []

for i in list_sorted_label:
    if i not in list_label:
        list_label.append(i)
        list_freq.append(list_sorted_label.count(i))

print(list_label)
print(list_freq)

[' ', 'a', 'e', 'f', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'x']
[7, 4, 4, 3, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1]


Afin de trier mes deux listes selon la liste des fréquences, j'utilise des tuples.

In [86]:
list_sorted_freq = zip(list_freq,list_label)
list_sorted_freq = sorted(list_sorted_freq)
list_sorted_freq = zip(*list_sorted_freq)

list_freq, list_label = [[*tuple] for tuple in list_sorted_freq]

print(list_freq)
print(list_label)

[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4, 4, 7]
['l', 'o', 'p', 'r', 'u', 'x', 'h', 'i', 'm', 'n', 's', 't', 'f', 'a', 'e', ' ']


Il est demandé d'afficher le nombre de caractères que contient notre nouvel alphabet.
J'affiche ensuite chaque caractère et sa fréquence associée comme demandé dans le sujet.

In [87]:
length_dict = len(list_label)
text_freq = str(length_dict) + "\n\n"

for i in range (length_dict):
    text_freq = text_freq + list_label[i] + " " + str(list_freq[i]) + "\n"

print(text_freq)

16

l 1
o 1
p 1
r 1
u 1
x 1
h 2
i 2
m 2
n 2
s 2
t 2
f 3
a 4
e 4
  7



J'écris mon dictionnaire dans un fichier en écrasant l'ancien contenu si le fichier existait déjà.

In [88]:
new_file = open(file_name +"/"+file_name + "_freq.txt", "w")
new_file.write(text_freq)
new_file.close()

Maintenant que j'ai mon dictionnaire, je peux commencer à compresser mon texte grâce à la technique de l'arbre de Huffman.

## Etape 2 : Construction de l’arbre 

Dans un premier temps, je crée une classe Node.
Je passe en paramètre :
- label = le caractère (a)
- freq = la fréquence de mon charactère dans le texte (4)
- le noeud enfant de gauche et le noeud enfant de droite si ces derniers existent, sinon ça veut dire que c'est une feuille.

In [89]:
class node:
    def __init__(self, label, freq, left_child=None, right_child=None):
         # Label de la lettre : "a"
        self.label = label
 
        # Fréquence de la lettre : 4
        self.freq = freq
 
        # L'enfant gauche(0) de la lettre en question : de type node
        self.left_child = left_child
 
        # L'enfant droit(1) de la lettre en question : de type node
        self.right_child = right_child
 
        # Direction : 0 ou 1
        self.direction = ''

    def __str__(self):
        return str(self.label) + " " + str(self.freq) + " " + str(self.left_child) + " " + str(self.right_child)

Pour créer l'algorithme, j'ai d'abord créé les feuilles.
Chaque caractère et sa fréquence sont donc une feuille que j'ajoute à une liste.

In [90]:
list_node = []

for i in range(len(list_label)):
    list_node.append(node(list_label[i], list_freq[i]))

for i in range(len(list_node)):
    print(list_node[i])

l 1 None None
o 1 None None
p 1 None None
r 1 None None
u 1 None None
x 1 None None
h 2 None None
i 2 None None
m 2 None None
n 2 None None
s 2 None None
t 2 None None
f 3 None None
a 4 None None
e 4 None None
  7 None None


L’algorithme  repose sur une structure d’arbre binaire où tous les nœuds internes ont exactement deux successeurs. 
Les feuilles sont étiquetées avec les caractères de l’alphabet.
Les branches sont étiquetées par 0 (fils gauche) et 1 (fils droit). 
Les chemins depuis la racine jusqu’aux feuilles constituent les codes des caractères.  

Je fais une boucle qui s'execute tant qu'on n'est pas à la racine de l'arbre.

In [91]:
while len(list_node) > 1:

    list_node = sorted(list_node, key=lambda x: x.freq)
 
    # Affectation des deux noeuds les plus petits
    node_left = list_node[0]
    node_right = list_node[1]
 
    # Affectation d'une direction à ces deux noeuds
    node_left.direction = 0
    node_right.direction = 1

    # La fréquence du parent devient la somme de la fréquence de ses deux enfants
    freq_parent = node_left.freq + node_right.freq

    # Enfin, on crée le noued parents avec les informations récoltées
    node_parent = node(freq_parent,freq_parent , node_left, node_right)
 
 
    # Suppression des noeuds enfants pour ne garder que le noeud parent
    list_node.remove(node_left)
    list_node.remove(node_right)
    list_node.append(node_parent)

print(list_node[0])

36 36 16 16 8 8 a 4 None None e 4 None None 8 8 4 4 h 2 None None i 2 None None 4 4 m 2 None None n 2 None None 20 20 8 8 4 4 s 2 None None t 2 None None 4 4 2 2 l 1 None None o 1 None None 2 2 p 1 None None r 1 None None 12 12 5 5 2 2 u 1 None None x 1 None None f 3 None None   7 None None


Comme on peut le voir, on obtient une liste d'un seul élément contenant tous les noeuds.
Pour créer l'arbre, il faut parcourir cet élément récursivement dans le but de créer le code binaire de chaque caractère.

In [92]:
def createListNodes(node, binary, list):
    # Créer le code binaire
    binary = binary + str(node.direction) 

    dict_comp = {}
    
    # Si le noeud contient un enfant gauche ou droit, alors ce n'est pas une feuille et il faut rappeler la fonction
    if (node.left_child):
        createListNodes(node.left_child, binary,list)

    if (node.right_child):
        createListNodes(node.right_child, binary,list)

    # Si le noeud n'a pas d'enfant, c'est une feuille.
    # On peut donc l'ajouter dans le dictionnaire et lui affecter son code binaire
    if(not node.left_child and not node.right_child):
        dict_comp[node.label] = binary
        # Je créer une liste de dictionnaire
        list.append(dict_comp)
        
    return list 

Voici donc à quoi ressemble la liste de dictionnaire après appel de la fonction.

In [93]:
binary = ""
list = []

list_nodes = createListNodes(list_node[0], binary,list )

print(list_nodes)

[{'a': '000'}, {'e': '001'}, {'h': '0100'}, {'i': '0101'}, {'m': '0110'}, {'n': '0111'}, {'s': '1000'}, {'t': '1001'}, {'l': '10100'}, {'o': '10101'}, {'p': '10110'}, {'r': '10111'}, {'u': '11000'}, {'x': '11001'}, {'f': '1101'}, {' ': '111'}]


On peut récupérer la clef, la valeur, ou les les deux de cette manière :

In [94]:
for key in list_nodes[0].keys():
    print (key)

for value in list_nodes[0].values():
    print (value)

for key,value in list_nodes[0].items():
    print(key, value)

a
000
a 000


## Etape 3 : Codage du texte

Je créer une liste qui contient les caractères de mon texte de base.

In [95]:
list_text = []

for i in text:
    list_text.append(i)
    
print(list_text)

['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 'a', 'n', ' ', 'e', 'x', 'a', 'm', 'p', 'l', 'e', ' ', 'o', 'f', ' ', 'a', ' ', 'h', 'u', 'f', 'f', 'm', 'a', 'n', ' ', 't', 'r', 'e', 'e']


Je parcours la liste de mon texte et je remplace chaque caractère par le code binaire qui correspond en parcourant ma liste de dictionnaire.

In [96]:
for i in range(len(list_text)):
    for j in list_nodes:
        for cle,valeur in j.items():
            if (cle == list_text[i]):
                list_text[i] = valeur
print(list_text)

['1001', '0100', '0101', '1000', '111', '0101', '1000', '111', '000', '0111', '111', '001', '11001', '000', '0110', '10110', '10100', '001', '111', '10101', '1101', '111', '000', '111', '0100', '11000', '1101', '1101', '0110', '000', '0111', '111', '1001', '10111', '001', '001']


Je concatène les éléments de ma liste pour obtenir une chaine de caractère.

In [97]:
text_binary = ""
for i in list_text:
    text_binary = text_binary + i

print(text_binary)

100101000101100011101011000111000011111100111001000011010110101000011111010111011110001110100110001101110101100000111111100110111001001


Pour que cette chaine soit lu en binaire, je la transforme.
J'ai pris une fonction sur internet qui permet de faire cela.

In [98]:
text_compressed = int(text_binary, base=2).to_bytes((len(text_binary)+7)//8, byteorder='big') 
print(text_compressed)

b'J,u\x8e\x1f\x9c\x86\xb5\x0f\xae\xf1\xd3\x1b\xac\x1f\xcd\xc9'


J'écris ce que j'obtiens dans un fichier.
Le "wb" permet de spécifier qu'on écrit en binaire.

In [99]:
new_file = open(file_name +"/"+file_name + "_comp.bin", "wb")
new_file.write(text_compressed)
new_file.close()

## Etape 4 : Détermination du taux de compression

Tout d'abord, je récupère les tailles de mon ancien fichier texte et mon nouveau fichier binaire.
Je calcul le taux de compression selon la formule qui m'est donnée.

In [100]:
text_compressed_size = os.path.getsize(file_name +"/"+file_name + "_comp.bin") 
print('Taille de '+ file_name + "_comp.bin" +' :', text_compressed_size, 'bits')

text_origin_size = os.path.getsize(file_name +"/"+file_name + '.txt') 
print('Taille de '+ file_name + ".txt" +' :', text_origin_size, 'bits')

amount = 1 - text_compressed_size/text_origin_size
print("Le taux est de :" ,round(amount,3))

Taille de textesimple_comp.bin : 17 bits
Taille de textesimple.txt : 36 bits
Le taux est de : 0.528


## Etape 5 : Détermination du nombre moyen de bits de stockage d'un caractère du texte compressé

Je fais une simple moyenne. 
Je prends chaque code binaire et je regarde le nombre de caractère qu'il y a (ex : 001 = 3; 1110 = 4).
J'additionne tous les chiffres et je divise cela par la longueur de ma list_text.

In [101]:
dividend = 0
diviseur = len(list_text)
for i in range(diviseur):
    dividend += len(list_text[i])

average = dividend/diviseur

print(average)

3.75
